In [17]:
!pip install fasttext
! pip install fasttext-langdetect

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for pybind11>=2.2 from https://files.pythonhosted.org/packages/06/55/9f73c32dda93fa4f539fafa268f9504e83c489f460c380371d94296126cd/pybind11-2.11.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 53.8 MB/s eta 0:00:00
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4282569 sha256=a484607de3ae501ef2fa23157f1c0e4856d7b2dcaa77afb1780d9b8705389fa3
  Stored in directory: /tmp/pip-ephem-wheel-cache-3u2935t8/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simp

In [18]:
import torch

In [19]:
torch.__version__

'2.0.0+cu117'

In [32]:
! pip install lightning-flash
! pip install 'lightning-flash[audio,text]'

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [20]:
! nvidia-smi

Thu Jul 20 20:20:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:1A:00.0 Off |                    0 |
| N/A   40C    P0    43W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:1C:00.0 Off |                    0 |
| N/A   

In [67]:
import pandas as pd
import numpy as np

In [68]:
song_metadata = pd.read_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/song_metadata_cleaned.csv", index_col= 0)

In [69]:
from ftlangdetect import detect

def is_english(text: str) -> str:
    """Simple Function to detect whether the language of the text is english or not.
    Returns a Boolean output for the same.Input the text column"""
    return detect(text, low_memory=True)["lang"]

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\%\$\&\^\*\@\#\<\>\/\+\\=\_\\}\{\)\(\]\[\`1234567890]'
song_metadata["transcription"] = song_metadata["transcription"].replace(chars_to_ignore_regex, '', regex=True)
song_metadata = song_metadata[song_metadata.transcription.str.len()>8]
song_metadata["language"] = song_metadata["transcription"].apply(is_english)
song_metadata = song_metadata.loc[song_metadata["language"] == "en"]
song_metadata

,file_name,transcription,language
90,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,betrogen kluge worte was,en
115,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,boy my life,en
116,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,ain't what it used to be,en
118,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,since you went out the door,en
119,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,all the times,en
...,...,...,...
250774,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,so take what i left you for the pain,en
250775,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,and do your best to forget my name,en
250782,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,chealternativeavrei,en
250786,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,do retta soloa me,en


In [70]:
train = song_metadata.sample(frac = 0.8, replace = False)
train.shape

(133328, 3)

In [71]:
inter_dataset = song_metadata.merge(train.drop_duplicates(), on=['file_name','transcription'], 
                   how='left', indicator=True)
validation = inter_dataset[inter_dataset["_merge"] == "left_only"][['file_name','transcription']]
validation

,file_name,transcription
8,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,the shakes the breaks
9,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,that rock the block
11,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,that spin the clock
14,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,the keys the chords
15,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,the beats the words
...,...,...
166645,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,and now that i'm gone
166646,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,try to forget me and just move on
166651,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,try to forget me and just move on
166652,/home/users/gmenon/dali/DALI_v1.0/audio/wav_cl...,i am not the one that you should blame


In [72]:
train.to_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_metadata_cleaned.csv")

In [73]:
validation.to_csv("/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_metadata_cleaned.csv")

In [74]:
for rows in validation["file_name"][33330:33331]:
    print(rows)

/home/users/gmenon/dali/DALI_v1.0/audio/wav_clips/ad887cbfa84749e5a9789f303e2f5c30.wav


In [75]:
for rows in validation["transcription"][33330:33331]:
    print(rows)

i am not the one that you should blame


In [76]:
import flash
import torch
from flash.audio import SpeechRecognition, SpeechRecognitionData
# from flash.core.data.utils import download_data
import pandas as pd
from dataclasses import dataclass

In [109]:
model = SpeechRecognition.load_from_checkpoint("/home/users/gmenon/workspace/songsLyricsGenerator/lightning_logs/version_1/checkpoints/epoch=0-step=3.ckpt")

trainer = flash.Trainer()

test_datamodule = SpeechRecognitionData.from_files(
    predict_files=["/home/users/gmenon/workspace/songsLyricsGenerator/src/notebooks/separated/mdx_extra/ad887cbfa84749e5a9789f303e2f5c30/vocals.wav",
                  "/home/users/gmenon/dali/DALI_v1.0/audio/wav_clips/ad887cbfa84749e5a9789f303e2f5c30.wav"], 
    batch_size=2)

wav2vec2_predictions = trainer.predict(model, datamodule=test_datamodule)

Using 'facebook/wav2vec2-base-960h' provided by Hugging Face/transformers (https://github.com/huggingface/transformers) and PyTorch/fairseq (https://github.com/pytorch/fairseq).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

In [110]:
wav2vec2_predictions

[['I AM NOT THEOWADAD YOU SIR BLAYY YOU', '']]